In [4]:
# import os
# current_dir = os.getcwd()
# print(current_dir)

/Users/tur92196/WorkDir/PRF_Ratios_syn/notebooks


In [26]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from typing import List, Tuple
from pandas import DataFrame
import pandas as pd
import matplotlib.pyplot as plt
from PRF_Ratios_syn import codon_analyses, data_processing

### DGRP2 data

In [ ]:
%cd ../data/dgrp2
!ls

In [92]:
main_df = pd.read_table('tables/NC_chr2L_tables.tsv',keep_default_na=True, na_values='NA')
# eij_annotations_df = pd.read_csv("extra_ann_tables/NC_Chr2L_extra_ann.tsv", keep_default_na=True, na_values='NA')

# phylop_scores_df = pd.read_csv('extra_ann_tables/dm6.phyloP27way.csv', sep=',')
# phastcons_scores_df = pd.read_csv('extra_ann_tables/dm6.27way.phastCons.csv', sep=',')

/var/folders/v5/5xxn2j4d4p1cqtkm5f1jx61xcmqqpm/T/ipykernel_77760/3006581971.py:1: DtypeWarning: Columns (26,27,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  main_df = pd.read_table('tables/NC_chr2L_tables.tsv',keep_default_na=True, na_values='NA')


In [97]:
main_df.iloc[:, [27]]

,lof_geneid
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
968207,NaN
968208,NaN
968209,NaN
968210,NaN


In [73]:
# Define swap pairs
swap_pairs = [
    ('ref', 'alt'),
    ('refcount', 'altcount'),
    ('refcontext', 'altcontext'),
    ('refcontext_complrev', 'altcontext_complrev'),
    ('refcodon', 'altcodon'),
    ('refaa', 'altaa')
]

In [74]:
def swap_values(row: DataFrame, swap_pairs: List[Tuple[str, str]]) -> DataFrame:
    """
    Function to swap values in a column.
    It is used to swap reference and alternative values in the .TSV files.
    It expects a list of tuples with the column names to swap.
    Swapping is performed in place, row by row, and in the whole dataframe
    using apply() function.
    """
    if row['aainfo'] == 'root_alt':
        for col1, col2 in swap_pairs:
            row[col1], row[col2] = row[col2], row[col1]
    return row

In [75]:
def create_codon_change(row: DataFrame) -> str:
    """
    Function to create codon change string.
    """
    ref = row['refcodon']
    alt = row['altcodon']
    if pd.isna(ref) or pd.isna(alt):
        return 'NA'
    return f"{ref}->{alt}"

In [76]:
def process_main_table(df: DataFrame, swap_pairs: List[Tuple[str, str]]):
    """
    Function to process main table.
    """
    df = df.apply(lambda row: swap_values(row, swap_pairs), axis=1)
    df['codon_change'] = df.apply(create_codon_change, axis=1)
    return df

In [78]:
# Merge with custom_annotation table
merged_df = pd.merge(main_df, eij_annotations_df, left_on=['chrom', 'pos'], right_on=['chrom', 'position'], how='left')

ValueError: You are trying to merge on int64 and object columns for key 'pos'. If you wish to proceed you should use pd.concat

In [30]:
def merge_tables(
    main_df: DataFrame,
    extra_annotation_df: DataFrame,
    phylop_scores_df: DataFrame,
    phastcons_scores_df: DataFrame
) -> DataFrame:
    """
    Function to merge tables.
    """
    
    # Merge with custom_annotation table
    merged_df = pd.merge(main_df, custom_annotation_df, left_on=['chrom', 'pos'], right_on=['chrom', 'position'], how='left')

    # Merge with score1 table
    merged_df = pd.merge(merged_df, phylop_df, left_on=['chrom', 'pos'], right_on=['chromosome', 'position'], how='left', suffixes=('', '_phylop'))

    # Merge with score2 table
    merged_df = pd.merge(merged_df, phastcons_df, left_on=['chrom', 'pos'], right_on=['chromosome', 'position'], how='left', suffixes=('', '_phastcons'))

    # Step 4: Clean up column names
    merged_df = merged_df.drop(columns=['chromosome', 'start', 'step', 'span', 'position'])
    merged_df = merged_df.rename(columns={'score_phylop': 'phyloP', 'score_phastcons': 'phastCons'})

    # Fill NaN values in custom_annotation, score1, and score2 columns with 'NA'
    columns_to_fill = ['custom_annotation', 'phyloP', 'phastCons']
    merged_df[columns_to_fill] = merged_df[columns_to_fill].fillna('NA')

    return merged_df

In [31]:
def process_chromosome(main_file, custom_annotations_file, phylop_file, phastcons_file, swap_pairs):
    # Read files
    main_df = pd.read_csv(main_file, sep='\t')
    custom_annotations_df = pd.read_csv(custom_annotations_file, sep='\t')
    phylop_df = pd.read_csv(phylop_file, sep='\t')
    phastcons_df = pd.read_csv(phastcons_file, sep='\t')

    # Process main table
    processed_df = process_main_table(main_df, swap_pairs)

    # Merge all dataframes
    merged_df = merge_tables(processed_df, custom_annotations_df, phylop_df, phastcons_df)

    return merged_df

In [38]:
main_df.head()

,chrom,pos,aainfo,ref,alt,qual,filtr,aa,ac,af,...,altaa,sift_trnscid,sift_geneid,sift_genename,sift_region,sift_vartype,sifts_core,sift_median,sift_pred,deleteriousness
0,chr2L,4985,root_unknown,C,A,.,PASS,C,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr2L,4990,root_unknown,T,G,.,PASS,T,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr2L,4998,root_unknown,G,A,.,PASS,G,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr2L,5039,root_unknown,C,T,.,PASS,C,8,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr2L,5040,root_unknown,G,A,.,PASS,G,6,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Define chromosome files
processed_chr2L = process_chromosome(
    'tables/NC_chr2L_tables.tsv',
    'extra_ann_tables/NC_Chr2L_extra_ann.tsv',
    'extra_ann_tables/dm6.phyloP27way.csv',
    'extra_ann_tables/dm6.27way.phastCons.csv',
    swap_pairs
)

# chromosome_files = [
#     ('tables/NC_chr2L_tables.tsv', 'extra_ann_tables/NC_Chr2L_extra_ann.tsv', 'extra_ann_tables/dm6.phyloP27way.csv', 'extra_ann_tables/dm6.27way.phastCons.csv')
# ]

/var/folders/v5/5xxn2j4d4p1cqtkm5f1jx61xcmqqpm/T/ipykernel_77760/1742752059.py:3: DtypeWarning: Columns (26,27,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  main_df = pd.read_csv(main_file, sep='\t')
/var/folders/v5/5xxn2j4d4p1cqtkm5f1jx61xcmqqpm/T/ipykernel_77760/1742752059.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  custom_annotations_df = pd.read_csv(custom_annotations_file, sep='\t')


KeyError: 'chromosome'

In [ ]:
processed_chr2L.head()

In [ ]:
# Save data
save_data(codon_data, 'codon_sfs_data.pickle', 'codon_sfs_data.json')
save_data(phylop_stats, 'phylop_stats.pickle', 'phylop_stats.json')
save_data(phastcons_stats, 'phastcons_stats.pickle', 'phastcons_stats.json')

In [ ]:
# Generate downsampled SFS
sample_sizes = [150, 100, 75, 50, 25]
downsampled_data = generate_downsampled_sfs(codon_data, sample_sizes)

In [ ]:
# Save downsampled data
save_data(downsampled_data, 'downsampled_sfs_data.pickle', 'downsampled_sfs_data.json')

In [ ]:
# Visualize phyloP and phastCons distributions
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

for codon_change, values in phylop_distribution.items():
    ax1.hist(values, bins=50, alpha=0.5, label=codon_change)
ax1.set_title('phyloP Score Distribution by Codon Change')
ax1.set_xlabel('phyloP Score')
ax1.set_ylabel('Frequency')
ax1.legend()

for codon_change, values in phastcons_distribution.items():
    ax2.hist(values, bins=50, alpha=0.5, label=codon_change)
ax2.set_title('phastCons Score Distribution by Codon Change')
ax2.set_xlabel('phastCons Score')
ax2.set_ylabel('Frequency')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Print summary statistics
print("phyloP Statistics:")
print(pd.DataFrame(phylop_stats).T)

print("\nphastCons Statistics:")
print(pd.DataFrame(phastcons_stats).T)